In [ ]:
import math

span = 1.95
spacing = 0.60
q_live = 3.0
q_dead = 0.15
density = 5.3
E_modulus = 11000e6
k_mod = 0.8
gamma_M = 1.3

beam_sizes = [(0.08, 0.16), (0.06, 0.12), (0.04, 0.08)]
classes = {"C24": 24, "C22": 22, "C18": 18}
f_vk = 4.0

for b, h in beam_sizes:
    beam_volume = b * h * span
    beam_weight = beam_volume * density
    w_surface = (q_live + q_dead) * spacing
    w_beam = beam_weight / span
    w_total = w_surface + w_beam
    M_max = w_total * span**2 / 8
    V_max = w_total * span / 2
    I = b * h**3 / 12
    c = h / 2
    sigma_max = M_max * c / I
    Q = b * h**2 / 8
    tau_max = V_max * Q / (I * b)
    A = b * h
    tau_avg = V_max / A
    delta_max = 5 * w_total * span**4 / (384 * E_modulus * I)
    print(f"\nBeam {int(b*100)}x{int(h*100)} cm")
    print(f"Distributed load: {w_total:.3f} kN/m")
    print(f"Max moment: {M_max:.3f} kN·m")
    print(f"Max shear: {V_max:.3f} kN")
    print(f"Bending stress: {sigma_max/1e6:.3f} MPa")
    print(f"Shear stress: {tau_max/1e6:.3f} MPa")
    print(f"Deflection: {delta_max*1000:.2f} mm")
    for cls, f_mk in classes.items():
        f_md = k_mod * f_mk / gamma_M
        f_vd = k_mod * f_vk / gamma_M
        print(f"{cls}: f_m,d={f_md:.2f} MPa, f_v,d={f_vd:.2f} MPa, OK? "
              f"Bend {'YES' if sigma_max/1e6<=f_md else 'NO'} | "
              f"Shear {'YES' if tau_max/1e6<=f_vd else 'NO'}")


In [ ]:
import numpy as np
np.linspace(0, 10, 5).tolist()[1:-1]

In [ ]:
span = 1.95
spacing = 0.60
q_live = 3.0
q_dead = 0.15
density = 5.3
k_mod = 0.8
gamma_M = 1.3
service_deflection_ratio = 300
dynamic_factors = [1.0, 1.2, 1.5]

beam_sizes = [(0.08, 0.16), (0.06, 0.12), (0.04, 0.08), (0.02, 0.04), (0.01, 0.02)]

wood_grades = {
    "C14 (Softwood)": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    "C18 (Softwood)": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    "C22 (Softwood)": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24 (Softwood)": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    "D30 (Hardwood)": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    "D40 (Hardwood)": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
    "D50 (Hardwood)": {"f_mk": 50, "f_vk": 5.0, "E_mean": 16000e6},
}

for b, h in beam_sizes:
    beam_volume = b * h * span
    beam_weight = beam_volume * density
    w_surface = (q_live + q_dead) * spacing
    defl_ok = True
    for dyn in dynamic_factors:
        if not defl_ok: break
        w_total = (w_surface * dyn + beam_weight / span) * 1000
        M_max = w_total * span**2 / 8
        V_max = w_total * span / 2
        I = b * h**3 / 12
        c = h / 2
        sigma_max = M_max * c / I
        Q = b * h**2 / 8
        tau_max = V_max * Q / (I * b)
        A = b * h
        tau_avg = V_max / A
        print(f"\nBeam {b*100:.1f}x{h*100:.1f} cm, Dynamic factor={dyn}")
        print(f"Distributed load: {w_total/1000:.3f} kN/m")
        print(f"Max moment: {M_max/1e3:.3f} kN·m")
        print(f"Max shear: {V_max/1e3:.3f} kN")
        print(f"Bending stress: {sigma_max/1e6:.3f} MPa")
        print(f"Shear stress: {tau_max/1e6:.3f} MPa")
        for grade, props in wood_grades.items():
            f_md = k_mod * props["f_mk"] / gamma_M
            f_vd = k_mod * props["f_vk"] / gamma_M
            E = props["E_mean"]
            delta_max = 5 * w_total * span**4 / (384 * E * I)
            service_limit = span / service_deflection_ratio
            bend_ok = sigma_max/1e6 <= f_md
            shear_ok = tau_max/1e6 <= f_vd
            defl_ok = delta_max <= service_limit
            status = []
            status.append("Bend OK" if bend_ok else "BEND FAIL")
            status.append("Shear OK" if shear_ok else "SHEAR FAIL")
            status.append("Defl OK" if defl_ok else "DEFLECTION FAIL")
            print(f"{grade}: f_m,d={f_md:.2f} MPa, f_v,d={f_vd:.2f} MPa, "
                  f"Defl={delta_max:.3f} m, Status: {', '.join(status)}")


In [ ]:
beam_base = 80
beam_height = 160
beam_length = 1910

beam_base**3 * beam_height * ((1/3) - (0.21 * beam_base/beam_height * (1 - (beam_base ** 4 / (12 * beam_height ** 4)))))

In [ ]:
0.229 * beam_length * beam_base ** 3

In [ ]:
import pandas as pd

# 1. Room & Floor Geometry
room_width_m = 1.87
room_length_m = 3.0

# 2. Loading Conditions
live_loads = range(3,31)
dead_load_planks_q_kPa = 0.25 # Dead load of the floor planks/decking in kN/m^2

# 3. Beam Configuration & Support
number_of_beams = 4
beam_bearing_length_m = 0.04

# 4. Material Properties & Safety Factors
k_mod = 0.8   # Modification factor for load duration and moisture (e.g., 0.8 for service class 2)
gamma_M = 1.3 # Partial safety factor for wood material properties
brick_safety_factor = 5.0 # Higher safety factor for brittle, hollow brick

# 5. Serviceability & Dynamic Limits
service_deflection_ratio = 360 # Stricter deflection limit (L/360 is common)
dynamic_factor = 1.0 # Assuming static load for simplicity. Change if floor will see dynamic use.

beam_sizes_m = [
    # (0.033, 0.07), 
    # (0.045, 0.09), 
    (0.06, 0.12), 
    (0.08, 0.16), 
    # (0.10, 0.20), 
    # (0.12, 0.24), 
]

wood_grades = {
    # "C14": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    # "C18": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    # "C22": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    # "D30": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    # "D40": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
}

wood_species = {
    # "Pinus sylvestris": {"density_kN": 4.707, "durability_DC": 3.5},
    # "Picea abies": {"density_kN": 4.413, "durability_DC": 4},
    "Larix decidua": {"density_kN": 5.884, "durability_DC": 3},
    # "Pinus radiata": {"density_kN": 4.217, "durability_DC": 4.5},
}

brick_types = {
    "Spanish Hollow Brick (Assumed)": {"compressive_strength_MPa": 10.0, "is_hollow": True},
    "Solid Clay Brick (for comparison)": {"compressive_strength_MPa": 20.0, "is_hollow": False},
}
selected_brick = brick_types["Spanish Hollow Brick (Assumed)"]

results = []
span = room_width_m
beam_spacing_m = room_length_m / number_of_beams

for b, h in beam_sizes_m:
    for live_load_q_kPa in live_loads:
        for grade_name, wood_props in wood_grades.items():
            for species_name, species_props in wood_species.items():
                # Self-weight of the beam
                beam_volume = b * h * span
                beam_weight_kN = beam_volume * species_props['density_kN']
                beam_self_weight_dist_kN_m = beam_weight_kN / span
                # Surface loads (live + dead) tributary to one beam
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * beam_spacing_m
                # Total design load on the beam
                w_total_kN_m = (surface_load_dist_kN_m + beam_self_weight_dist_kN_m) * dynamic_factor
                w_total_N_m = w_total_kN_m * 1000

                # Bending and Shear Forces
                M_max_Nm = w_total_N_m * span**2 / 8
                V_max_N = w_total_N_m * span / 2 # This is also the reaction force at the support
                # Beam geometric properties
                I_m4 = b * h**3 / 12  # Moment of inertia
                A_m2 = b * h           # Cross-sectional area
                # Stresses in the beam
                sigma_max_Pa = (M_max_Nm * (h / 2)) / I_m4
                tau_max_Pa = (3 * V_max_N) / (2 * A_m2) # Max shear for rectangle
                # Allowable design stresses for the wood
                f_md_Pa = (k_mod * wood_props["f_mk"] / gamma_M) * 1e6
                f_vd_Pa = (k_mod * wood_props["f_vk"] / gamma_M) * 1e6
                # Deflection
                E_Pa = wood_props["E_mean"]
                delta_max_m = (5 * w_total_N_m * span**4) / (384 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio

                # Calculate stress on bricks
                bearing_area_m2 = b * beam_bearing_length_m
                bearing_stress_on_brick_Pa = V_max_N / bearing_area_m2
                allowable_brick_stress_Pa = (selected_brick["compressive_strength_MPa"] * 1e6) / brick_safety_factor

                # Failure mode booleans
                bend_ok = sigma_max_Pa <= f_md_Pa
                shear_ok = tau_max_Pa <= f_vd_Pa
                defl_ok = delta_max_m <= service_limit_m
                brick_ok = bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa

                results.append({
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Wood Grade": grade_name,
                    "Wood Species": species_name,
                    "Total Beam Weight": beam_weight_kN * 6,
                    "Beam Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6,
                    "Beam Shear Stress (MPa)": tau_max_Pa / 1e6,
                    "Allowable Shear (MPa)": f_vd_Pa / 1e6,
                    "Live load": live_load_q_kPa,
                    "Deflection (mm)": delta_max_m * 1000,
                    "Allowable Defl (mm)": service_limit_m * 1000,
                    "Brick Bearing Stress (MPa)": bearing_stress_on_brick_Pa / 1e6,
                    "Allowable Brick Stress (MPa)": allowable_brick_stress_Pa / 1e6,
                    "Bend OK": bend_ok,
                    "Shear OK": shear_ok,
                    "Defl OK": defl_ok,
                    "Brick OK": brick_ok
                })

df = pd.DataFrame(results)
df

In [ ]:
from Pynite import FEModel3D
from Pynite.Rendering import Renderer
frame = FEModel3D()

# Constants
room_width = 1870
room_length = 3000
floor_height = 2750
beam_base = 60
beam_height = 120
wall_beam_contact_area = 40
beam_length = room_width + wall_beam_contact_area # NOT full length, calculated brick' center. Full length == room_width + wall_beam_contact_area * 2

# Materials
E = 11000 # MPa (N/mm²)
nu = 0.3
G = E / (2 * (1 + nu))  # MPa (N/mm²)
rho = 6.0e-7  # kg/mm3
frame.add_material('wood', E=E, G=G, nu=nu, rho=rho)

E = 7000 # MPa (N/mm²)
nu = 0.2
G = E / (2 * (1 + nu))  # MPa (N/mm²)
rho = 5.45e-7 # kg/mm3
frame.add_material('brick', E=E, G=G, nu=nu, rho=rho)

# Beam cross-section
A = beam_base * beam_height
J = beam_base ** 3 * beam_height * ((1/3) - (0.21 * beam_base/beam_height * (1 - (beam_base ** 4 / (12 * beam_height ** 4)))))
Iy = (beam_base ** 3 * beam_height) / 12
Iz = (beam_base * beam_height ** 3) / 12
frame.add_section('beam', A, Iy, Iz, J)

# Floor nodes and supports
frame.add_node('floor SE', beam_base/2, 0, 0)
frame.add_node('floor NE', beam_base/2, 0, room_width)
# frame.add_node('floor SW', room_length - beam_base/2, 0, 0)
# frame.add_node('floor NW', room_length - beam_base/2, 0, room_width)

frame.add_member('A', 'floor SE', 'floor NE', 'wood', 'beam')

In [ ]:
frame.members['A'].section.name

In [ ]:
# Constants
room_width = 1870
room_length = 3017
room_height = 5465
plank_thickness = 25
floor2floor = room_height/2 + plank_thickness/2
floor2ceiling = room_height/2 - plank_thickness/2
beam_base = 60
beam_height = 160
floor2beam = floor2ceiling - beam_height
wall_beam_contact_area = 40
beam_length = room_width + wall_beam_contact_area

In [ ]:
floor2ceiling, floor2floor, floor2beam

In [ ]:
E = 11000 # MPa (N/mm²)
nu = 0.3
G = E / (2 * (1 + nu))  # MPa (N/mm²)
G